In [ ]:
import opengeode as og 
import opengeode_geosciences as ogg
import geode_numerics
import geode_implicit 

# define various utilities functions
import os
def input(path):
  return os.path.join("data/",path) 

def create_folder_if_does_not_exists(path):
  if( not os.path.isdir(path)):
    os.makedirs(path)

def output(path):
  result_path = "results/"
  create_folder_if_does_not_exists(result_path)
  return os.path.join(result_path,path)

def create_aligned_box3D(extent_x,extent_y,extent_z):
    bbox = og.BoundingBox3D()
    bbox.add_point(og.Point3D([0,0,0]))
    bbox.add_point(og.Point3D([extent_x,extent_y,extent_z]))
    return bbox

print("Geode module imported!")

In this tutorial, e introduce the Geode implicit modeling tool to create a structural model boundary representation.
 * Create the volume of interest
 * Set up scalar field contraints
 * Interpolate scalar field
 * Create Structural Model Boundary Representation

Visualizations will be done in paraview. 

## Define the volume of interest 

In [ ]:
box_voi = create_aligned_box3D(40.,40.,40.)

print("An aligned box have been created lower left coordinates ("+
      str(box_voi.min().value(0))+"," +str(box_voi.min().value(1))+","+str(box_voi.min().value(2))+
      ") and upper right coordinates ("+str(box_voi.max().value(0))+"," +str(box_voi.max().value(1))+
      ","+str(box_voi.max().value(2))+")."
      )

## Set up scalar field constraints 
Field contraints are defined to create a moniclinal structural model. 

In [ ]:
# input point set file
monoclinal_point_set_filename = input(
            "00_monoclinal_implicit_data_constraints3D.og_pts3d")

# convert data file to vtp file format 
pset_data = og.load_point_set3D(monoclinal_point_set_filename)
og.save_point_set3D(pset_data,output(
            "00_monoclinal_implicit_data_constraints3D.vtp"))

# Build the data point manager and set up values from a file.
data_constraints = geode_numerics.DataPointsManager3D()
data_constraints.load_data_points(monoclinal_point_set_filename)

print(str(data_constraints.nb_data_points())+" points have been registered to constrain the scalar field.")

Points can be seen opening the ''file.vtp'' in paraview.

## Interpolate the scalar field.
The interpolation of the scalar field from the data points is done using the finite difference method. Such a method requires a structured grid for the computation. The resolution of this grid is a tradeoff between the resolution of the available data and the computational efficiency.

Please refert to [ScalarFunctionComputer](https://docs.geode-solutions.com/references/geode-implicit/ScalarFunctionComputer.html).

In [ ]:
# initialize the scalar field interpolator 
grid_step_vector = 4
function_computer = geode_implicit.ScalarFunctionComputer3D(data_constraints, box_voi, grid_step_vector)

# compute the scalar field
scalar_field_property_name = "boundaryfree_curvature_min"
function_computer.compute_scalar_function(scalar_field_property_name)

## Create structural model boundary representation
Extract an explicit representation of isovalues using the [GridScalarFunctionExpliciter3D](https://docs.geode-solutions.com/references/geode-implicit/GridScalarFunctionExpliciter3D.html). 

In [ ]:
# define scalar field isovalue
nb_isovalues = 5
isovalues = [2. / (10 - isovalue_id) for isovalue_id in range(nb_isovalues)]

# set up expliciter tool
expliciter = geode_implicit.GridScalarFunctionExpliciter3D(
        function_computer.grid_with_results(), scalar_field_property_name)
expliciter.add_scalar_isovalues(isovalues)

# export the boundary representation
brep = expliciter.build_brep()
brep_filename = output("00_monoclinal_brep.vtm")
og.save_brep(brep, brep_filename)

The boundary representation is now enriched with geological information and tha proper structural model is built. 

In [ ]:
structural_model = ogg.StructuralModel(brep)
    
implicit_model = ogg.implicit_model_from_structural_model_scalar_field(
        structural_model, scalar_field_property_name)

structural_model_filename = output("00_monoclinal_structural_model.og_istrm")
ogg.save_implicit_structural_model(
        implicit_model, structural_model_filename)

The model can be edited:
 * modifying constraining data
 * reinterpolating scalar field
 * exporting new boundary representation

In [ ]:
def change_data_constraints(data_constraints):
    data_constraints.add_data_point(
        og.Point3D([5., 6.25, 34.]), 2, 10)
    data_constraints.add_data_point(
        og.Point3D([29.5, 30.3, 36.5]), 2, 10)
    data_constraints.add_data_point(
        og.Point3D([12.1, 24.9, 35.]), 2, 10)

change_data_constraints(data_constraints)
modified_pset_data = output(
        "00_monoclinal_implicit_data_constraints3D_v2.vtp")
data_constraints.save_data_points_manager(modified_pset_data)

modified_scalar_field_property_name = "boundaryfree_curvature_min_v2"
function_computer.compute_scalar_function_with_value_preconditioning(
        modified_scalar_field_property_name, scalar_field_property_name)

# set up expliciter tool
modifyied_expliciter = geode_implicit.GridScalarFunctionExpliciter3D(
        function_computer.grid_with_results(), modified_scalar_field_property_name)
modifyied_expliciter.add_scalar_isovalues(isovalues)

# export the boundary representation
modifyied_brep = modifyied_expliciter.build_brep()
modifyied_brep_filename = output("00_monoclinal_brep_v2.vtm")
og.save_brep(modifyied_brep, modifyied_brep_filename)

Observe model deformation when adding the three points in red.
![model comparison](./pictures/model_comp.png)